# 1D Convolutional Variational Autoencoder

*   Input data shape (12906, 658) encoded with corresponding integers.

*   The training accuracy with 100 epochs can lead to 97%. The following training accuracy with 60 epochs is up to 96%.




In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
## adjust the length of DNA sequence with 658
## remove the uncorrect characters 'K','M', 'N', 'R', 'S', 'W', 'Y'
df = pd.read_csv (r'/content/train_features.csv')
def equal_length(seq):
    seq2 = seq.ljust(1058, '-') 
    fir = seq2[0:658]
    remove_characters = ['K','M', 'N', 'R', 'S', 'W', 'Y']
    for character in remove_characters:
        fir = fir.replace(character, "-")
    return fir
dna = df.dna.apply(equal_length)
## Example for arbitery DNA sequence
dna[3]

'AACATTATATTTTATTTTTGGTGCATGAGCTGGAATAGTAGGAACTTCATTAAGTATGCTAATTCGAGCTGAATTAGGAAACCCAGGTGCATTAATTGGAGATGACCAAATTTATAATGTTATTGTTACTGCACATGCATTTATTATGATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTGGAAATTGATTAGTTCCTTTAATATTAGGAGCCCCAGATATAGCTTTTCCTCGAATAAATAATATAAGTTTTTGATTACTTCCTCCTTCATTAACGCTCCTCTTAATAAGAAGTCTAGTG---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------'

In [3]:
## Transfer Character to Int
from numpy import argmax
def num_encoder(seq):
  alphabet = 'ACGT-'
  char_to_int = dict((c, i) for i, c in enumerate(alphabet))
  to_array = [char_to_int[char] for char in seq]
  return(np.array(to_array).reshape((658)))

dna_code = np.array(dna.apply(num_encoder).to_list())
## Example
dna_code[3]

array([0, 0, 1, 0, 3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 2, 2, 3,
       2, 1, 0, 3, 2, 0, 2, 1, 3, 2, 2, 0, 0, 3, 0, 2, 3, 0, 2, 2, 0, 0,
       1, 3, 3, 1, 0, 3, 3, 0, 0, 2, 3, 0, 3, 2, 1, 3, 0, 0, 3, 3, 1, 2,
       0, 2, 1, 3, 2, 0, 0, 3, 3, 0, 2, 2, 0, 0, 0, 1, 1, 1, 0, 2, 2, 3,
       2, 1, 0, 3, 3, 0, 0, 3, 3, 2, 2, 0, 2, 0, 3, 2, 0, 1, 1, 0, 0, 0,
       3, 3, 3, 0, 3, 0, 0, 3, 2, 3, 3, 0, 3, 3, 2, 3, 3, 0, 1, 3, 2, 1,
       0, 1, 0, 3, 2, 1, 0, 3, 3, 3, 0, 3, 3, 0, 3, 2, 0, 3, 3, 3, 3, 3,
       3, 3, 3, 0, 3, 0, 2, 3, 3, 0, 3, 0, 1, 1, 3, 0, 3, 3, 0, 3, 0, 0,
       3, 3, 2, 2, 0, 2, 2, 0, 3, 3, 3, 2, 2, 0, 0, 0, 3, 3, 2, 0, 3, 3,
       0, 2, 3, 3, 1, 1, 3, 3, 3, 0, 0, 3, 0, 3, 3, 0, 2, 2, 0, 2, 1, 1,
       1, 1, 0, 2, 0, 3, 0, 3, 0, 2, 1, 3, 3, 3, 3, 1, 1, 3, 1, 2, 0, 0,
       3, 0, 0, 0, 3, 0, 0, 3, 0, 3, 0, 0, 2, 3, 3, 3, 3, 3, 2, 0, 3, 3,
       0, 1, 3, 3, 1, 1, 3, 1, 1, 3, 3, 1, 0, 3, 3, 0, 0, 1, 2, 1, 3, 1,
       1, 3, 1, 3, 3, 0, 0, 3, 0, 0, 2, 0, 0, 2, 3,

In [4]:
dna_code.shape

(12906, 658)

In [6]:
from tensorflow.python.keras.layers.core import Flatten
y_train = pd.read_csv (r'/content/train_labels.csv')
ytrain = np.array(y_train['labels'], dtype = "uint32")
xtrain = np.expand_dims(dna_code, -1)
model = tf.keras.models.Sequential([  
  keras.Input(shape=(658,1)),
  layers.Conv1D(258, 3, activation="relu", padding="same"),
  layers.Conv1D(64, 3, activation="relu", padding="same"),
  tf.keras.layers.Flatten(),
  layers.Dense(518, activation='relu'),
  tf.keras.layers.Dense(1220, activation='softmax')
])

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(xtrain, ytrain, epochs=60, validation_split=0.3, verbose=2)

Epoch 1/60
283/283 - 8s - loss: 7.0347 - accuracy: 0.0108 - val_loss: 6.9336 - val_accuracy: 0.0207
Epoch 2/60
283/283 - 7s - loss: 6.6849 - accuracy: 0.0469 - val_loss: 6.5279 - val_accuracy: 0.0666
Epoch 3/60
283/283 - 7s - loss: 5.4314 - accuracy: 0.1843 - val_loss: 6.5272 - val_accuracy: 0.1051
Epoch 4/60
283/283 - 7s - loss: 2.9895 - accuracy: 0.4817 - val_loss: 6.0366 - val_accuracy: 0.1420
Epoch 5/60
283/283 - 7s - loss: 1.0314 - accuracy: 0.7980 - val_loss: 5.3719 - val_accuracy: 0.1116
Epoch 6/60
283/283 - 7s - loss: 0.7058 - accuracy: 0.8768 - val_loss: 0.6833 - val_accuracy: 0.8802
Epoch 7/60
283/283 - 7s - loss: 0.2301 - accuracy: 0.9489 - val_loss: 0.5855 - val_accuracy: 0.9032
Epoch 8/60
283/283 - 7s - loss: 0.1467 - accuracy: 0.9680 - val_loss: 0.7450 - val_accuracy: 0.8732
Epoch 9/60
283/283 - 7s - loss: 0.0995 - accuracy: 0.9811 - val_loss: 2.4078 - val_accuracy: 0.4533
Epoch 10/60
283/283 - 7s - loss: 0.3529 - accuracy: 0.9586 - val_loss: 0.4865 - val_accuracy: 0.9396